## <span style="color:grey">Initialization</span>

In [1]:
try:
    mode
except Exception as e:
    %run "KiteConnect_Library.ipynb"  
    logging.debug(e)
    mode = "Algo"
    papertrade = True


### Filter Stocks - portfolio maker

In [2]:
if mode == "Algo":
    portfolio = ['HDFCBANK']

##  <span style="color:green">Trading Strategy </span>

In [42]:
# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
VOLUME = 0
BBT = 0
BBM = 0
BBB = 0
AroonUp = 0
AroonDown = 0
iMACD = 0
iMACDhist = 0
iMACDsignal = 0
UP = 0
DOWN = 1
SD = 0
iRSI = 0 
atr = 0
STATE = "SELL"
sma20 = 0
sma50 = 0
buy_df = pd.DataFrame()
sell_df = pd.DataFrame()

# Methods
REF = lambda df, i: df.shift(i)
#TREND_UP = lambda a, days: ROC(a,days) >= 0.01
#TREND_DOWN = lambda a, days: ROC(a,days) <= -0.01

TREND_UP = lambda high, low: ((REF(high,1) >= REF(high,2)) & (REF(low,1) >= REF(low,2)))
TREND_DOWN =  lambda high, low: ((REF(high,1) <= REF(high,2)) & (REF(low,1) <= REF(low,2)))
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)
profit = 0
stoploss = 0
squareoff = 0

class algoTrade:
    
    def __init__(s, price):
        #logging.debug(price.tail(10))
        global OPEN, CLOSE, HIGH, LOW, VOLUME, BBT, BBM, BBB, AroonDown, AroonUp
        global iMACD, iMACDhist, iMACDsignal, SD, iRSI, sma20, sma50
        OPEN = price['open']
        CLOSE = price['close']
        HIGH = price['high']
        LOW = price['low']
        VOLUME = price['volume']
        BBT, BBM, BBB = BBANDS( CLOSE, 20,2,2,1)
        iMACD, iMACDsignal, iMACDhist = MACDEXT(CLOSE, fastperiod=12, slowperiod=26, signalperiod=9,  fastmatype=1, slowmatype=1,signalmatype=1)
        SD = STDDEV(CLOSE)
        sma20 = SMA(CLOSE, timeperiod=20)
        sma50 = SMA(CLOSE, timeperiod=50)
    
    # Long Strategies
    def long_bb(self):
        return pd.DataFrame( (REF(CLOSE, 1) < REF(BBB, 1)) & (CLOSE > BBB), columns=["buy"] )
     
    # Short Strategies
    def short_bb(self):
        return pd.DataFrame((REF(CLOSE,1) > REF(BBT,1)) & (CLOSE<BBT), columns=["sell"])
    
    
    def long_pa(self):
        return pd.DataFrame(TREND_DOWN(HIGH, LOW) & 
                            ((CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100)
                            | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == 100)
                            #| (CDLABANDONEDBABY(OPEN, HIGH, LOW, CLOSE) != 0) 
                            ), columns=["buy"])
    def short_pa(self):
        return pd.DataFrame( TREND_UP(HIGH, LOW) & 
                            ((CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100)
                            | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == -100)
                            #| (CDLEVENINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0)
                            ) , columns=["sell"])

    
    # do not set timestamp for live trading
    def tradeBot(self, symbol, timestamp=dt.datetime.now().replace(second=0, microsecond=0)):
        global STATE, profit, squareoff, stoploss
        
        timeNow = timestamp        
        delT1 = timeNow - dt.timedelta(minutes = 1)
        delT2 = timeNow - dt.timedelta(minutes = 2)

        price = CLOSE[-1]
        trigger = CLOSE[-1]
        stoploss = price - 2 * SD[-1] 
        squareoff = 3 * SD[-1] 

        check_trade = lambda x, dt1, dt2: True if (x.shape[0] > 0) and ((x.index[-1]==dt1) or (x.index[-1]==dt2)) else False
        print_trade = lambda x: logging.debug(x) if (x.empty != True) else False
        
        buy = pd.DataFrame()
        sell = pd.DataFrame()
        
        if (timeNow.time() > dt.time(11,30)) and (timeNow.time() < dt.time(14,0)):
            # Mostly Choppy period# Mostly Trending period
            if STATE == "BUY":
                buy  = self.long_bb()
                buy = buy[buy['buy']]
                #print_trade(buy)
                #STATE = "SELL"
            elif STATE == "SELL":
                sell = self.short_bb()
                sell = sell[sell['sell']]
                #print_trade(sell)   
                #STATE = "BUY"    
        else:
            # Mostly Trending period
            if STATE == "BUY":
                buy  = self.long_pa()
                buy = buy[buy['buy']]
                #print_trade(buy)
                #STATE = "SELL"
            elif STATE == "SELL":
                sell = self.short_pa()
                sell = sell[sell['sell']]
                #print_trade(sell)   
                #STATE = "BUY"    
            
        if check_trade(buy, delT1, delT2):            
            logger.debug("buy:"+str(buy.index[-1]))
            
            ##### Order Placement #####
            #buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            buy_slm(symbol, price, stoploss, quantity=1)
            STATE = "SELL"
            return -1 * price
            
        elif check_trade(sell, delT1, delT2):                
            logger.debug("sell:"+str(sell.index[-1]))
            
            ##### Order Placement #####
            #sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            sell_slm(symbol, price, stoploss, quantity=1)
            STATE = "BUY"
            return price
        
        return 0
    
def backtest(portfolio, fromDate, toDate):
    global STATE, buy_df, sell_df
    offset = 90
    for index,symbol in portfolio.iterrows():
        profit = 0
        STATE = "BUY"
        squareoff = 0
        stoploss = 0
        global fig
        noofcandles = 60
        x = symbol[0]
        logging.debug(str(fromDate) + "," + str(toDate))
        temp_data = getData(x, fromDate = fromDate, toDate=toDate, freq="minute")
        rowSize = temp_data.shape[0]

        for i in np.linspace(offset,rowSize-1, rowSize-offset -1, dtype="int64"):
            if(toDate.date() != temp_data.index[i].date()):
                continue
            a = algoTrade(temp_data.iloc[(i-offset):i])
            
            temp_profit = a.tradeBot(x, temp_data.index[i].to_pydatetime())
            
            profit = profit + temp_profit
            
            if temp_profit > 0:
                buy_df = buy_df.append(temp_data.iloc[i])
            elif temp_profit < 0:
                sell_df = sell_df.append(temp_data.iloc[i])
            
            
            #print(temp_data.iloc[(i-30):i])
        
        if STATE == "SELL":
            profit = profit + temp_data.iloc[-1].close

        print(x+": "+str(round(profit,2))+", ("+str(round(profit/temp_data.iloc[-1].close*100,4))+" %)")
        logging.debug(x+": "+str(round(profit,2))+", ("+str(round(profit/temp_data.iloc[-1].close*100,4))+" %)")    


#portfolio = pd.DataFrame(downloadlist)
if mode == "Algo":
    toDate = dt.datetime(2019,4,16,15,15)
    fromDate = toDate - dt.timedelta(days = 2, hours=4)
    STATE = "BUY"
    logger.setLevel(logging.DEBUG)
    portfolio = pd.DataFrame(['WIPRO'])
    papertrade = True
    
    backtest(portfolio, fromDate, toDate)
    gc.collect()
    
    x = 'WIPRO'
    temp_data = getData(x, fromDate = fromDate, toDate=toDate, freq="minute")
    fig = createPlot(x)
    plot_set1(temp_data)
    annotateBuySell(buy_df, "Buy")
    annotateBuySell(sell_df, "Sell")
    plotData(x, True)

WIPRO: -7.0, (-2.4929 %)
This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



- Lunch Time: 11:00 am to 2:00 pm => Choppy Market
- Bollinger band ZONE: for breakout

Choppy market Trading:
- Identify stocks with highest SD and trade using bollinger

Trending Market Trading:
- MACD or price action trading based on candle stick